# 📶 Telecomunicaciones: churn, ARPU y calidad de red
KPIs: churn, ARPU, NPS, disponibilidad, fallas por sitio.

## Casos de uso
- Predicción de churn y ofertas de retención
- Optimización de celdas y capacidad
- Detección de fraude en recargas y SIM swapping
- SLA por cliente y reporte regulatorio

In [ ]:
# Mini cálculo ARPU
ingresos = [10, 12, 8, 15]
subs = 4
sum(ingresos)/max(subs,1)

## Arquitectura
- CDRs en streaming, enriquecimiento con GIS
- Medición QoE/QoS y SLA por segmento
- Feature store + modelos de churn y capacity planning

## 💼 Valor en Telco
- Menor churn con retención proactiva y ofertas personalizadas.
- ARPU ↑ mediante upsell/cross-sell y optimización de capacidad.
- SLA y QoE transparentes por segmento/cliente.


In [ ]:
# Práctica: propensión de churn (toy)
import pandas as pd
import math

df = pd.DataFrame({
    'tenure_meses':[1,3,6,12,24,36],
    'uso_gb':[1.2, 3.5, 5.0, 7.1, 10.3, 12.5],
    'reclamos_90d':[2,1,1,0,0,0]
})
# score simple: más reclamos y poco tenure → mayor churn
z = -1.5 + 0.1*df['reclamos_90d'] - 0.02*df['tenure_meses'] - 0.01*df['uso_gb']
df['churn_prob'] = 1/(1+(-z).apply(math.exp))
print(df)


## 🏗️ Arquitectura CDR/GIS/SLA
- CDRs en streaming + enriquecimiento GIS → medición QoE y congestión.
- Feature store para churn y capacity planning.
- APIs de SLA por cliente y reportes regulatorios.
